In [1]:
import numpy as np
import pandas as pd
import time
from dobbel import dobbellogger
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
def expq(vector, multiplier=1):
    x, y, z = vector[0] * multiplier, vector[1] * multiplier, vector[2] * multiplier
    absq = (x ** 2 + y ** 2 + z ** 2) ** 1 / 2
    w = np.cos(absq)
    sinabs = np.sin(absq)
    xq = x * sinabs / absq
    yq = y * sinabs / absq
    zq = z * sinabs / absq
    return np.array([w, xq, yq, zq])


def quat_mul(p, q):
    p0 = p[0]
    q0 = q[0]
    pv = p[1:]
    qv = q[1:]
    out0 = p0 * q0 - np.dot(pv, qv)
    outv = p0 * qv + q0 * pv + np.cross(pv, qv)
    out = np.array([out0, *outv])
    return out

def quaternion_conjugate(q):
    w, x, y, z = q
    return np.array([w, -x, -y, -z])


# Actual Rotation Fuction - Input = Acceleration vector = 1x3 , rotation_quaternion = 1*4
def rotate_vector(vector, rotation_quaternion):
    # Convert the vector to a quaternion
    vector_quaternion = np.concatenate(([0], vector))

    # Calculate the rotated quaternion
    rotated_quaternion = quat_mul(rotation_quaternion,
                                  quat_mul(vector_quaternion, quaternion_conjugate(rotation_quaternion)))

    # Extract the rotated vector from the quaternion
    rotated_vector = rotated_quaternion[1:]

    return np.array(rotated_vector)

def remove_nan(df):
    counter = 0
    nan = True
    while nan:
        if not df.loc[counter].isna().any():
            nan = False
        counter += 1
    for i in range(counter - 1):
        df = df.drop(index=i)
    df = df.reset_index(drop=True)
    counter = len(df) - 1
    nan = True
    while nan:
        if not df.loc[counter].isna().any():
            nan = False
        counter -= 1
    maxlen = len(df)
    for i in range(maxlen - 1, counter + 1, -1):
        df = df.drop(index=i)
    df = df.reset_index(drop=True)
    for i in range(len(df)):
        df['timestamp'][i] = df['timestamp'][i] - df['timestamp'][0]
    for i in range(len(df)):
        for column in df.columns:
            if np.isnan(df[column][i]):
                deler = (df['timestamp'][i] - df['timestamp'][i - 1]) / (df['timestamp'][i + 1] - df['timestamp'][i - 1])
                df[column][i] = df[column][i - 1] + deler * (df[column][i + 1] - df[column][i - 1])
    return df

In [3]:
angle = np.deg2rad(-39)
rotate_y = np.array([[np.cos(angle),0,np.sin(angle)], [0,1,0],[-np.sin(angle),0,np.cos(angle)]])
trace = np.trace(rotate_y)
q_0 = np.sqrt(1+trace)/2
q_2 = 1/(4*q_0) * (np.sin(angle)+np.sin(angle))
q_rotate = np.array([q_0,0,q_2,0])
print(q_rotate)

[ 0.94264149  0.         -0.33380686  0.        ]


In [4]:
# Create rotated accelerometer lists
xr_acc_list = []
yr_acc_list = []
zr_acc_list = []
xr_acc_stds_list = []
yr_acc_stds_list = []
zr_acc_stds_list = []

# Create rotated gyroscope lists
xr_gyro_list = []
yr_gyro_list = []
zr_gyro_list = []
xr_gyro_stds_list = []
yr_gyro_stds_list = []
zr_gyro_stds_list = []

dob = dobbellogger()

for i in range(3):
    i=i+1
    print("Leg de dobbelsteen met nummer ", i, " boven")
    print("wacht 3 secondes")
    time.sleep(3)
    dob.connect()

    #Log the data of the dice
    dob.log(2, 200, 8, 125)
    
    # Download the logged data from the dice
    dob.download()
    data = dob.datadf
    data = remove_nan(data)

    # Calculate the means of the accelerometer output
    x_acc = np.mean(data['x_acc'])
    y_acc = np.mean(data['y_acc'])
    z_acc = np.mean(data['z_acc'])
    acc_means = np.array([x_acc,y_acc,z_acc])
  
    #Calculate the standard deviations of the accelerometer output
    x_std = np.std(data['x_acc'])
    y_std = np.std(data['y_acc'])
    z_std = np.std(data['z_acc'])
    acc_stds = np.array([x_std, y_std, z_std])

    #Rotate the means and standard deviations of the accelerometer to the initial triad
    acc_means_init = rotate_vector(acc_means, q_rotate)
    acc_stds_init = rotate_vector(acc_stds, q_rotate)
    
    #Append the rotated means of the accelerometer to the list
    xr_acc_list.append(acc_means_init[0])
    yr_acc_list.append(acc_means_init[1])
    zr_acc_list.append(acc_means_init[2])
    
    #Append the rotated standard deviations of the accelerometer to the list
    xr_acc_stds_list.append(acc_stds_init[0])
    yr_acc_stds_list.append(acc_stds_init[1])
    zr_acc_stds_list.append(acc_stds_init[2])
    
    #Create DataFrames from the list of rotated accelerometer means and standard deviations
    acc_means_init_df = pd.DataFrame({"xr_acc": xr_acc_list, "yr_acc": yr_acc_list, "zr_acc": zr_acc_list})
    acc_stds_init_df = pd.DataFrame({"xr_std": xr_acc_stds_list, "yr_std": yr_acc_stds_list, "zr_std": zr_acc_stds_list})
    
    # Calculate the means of the gyroscope output
    x_gyro = np.mean(data['x_gyro'])
    y_gyro = np.mean(data['y_gyro'])
    z_gyro = np.mean(data['z_gyro'])
    gyro_means = np.array([x_gyro, y_gyro, z_gyro])
    
    # Calculate the standard deviations of the gyroscope output
    x_gyro_std = np.std(data['x_gyro'])
    y_gyro_std = np.std(data['y_gyro'])
    z_gyro_std = np.std(data['z_gyro'])
    gyro_stds = np.array([x_gyro_std, y_gyro_std, z_gyro_std])

    #Rotate the means and standard deviations of the accelerometer to the initial triad
    gyro_means_init = rotate_vector(gyro_means, q_rotate)
    gyro_stds_init = rotate_vector(gyro_stds, q_rotate)
    
    # Append the rotated means of the gyroscope output to the list
    xr_gyro_list.append(gyro_means_init[0])
    yr_gyro_list.append(gyro_means_init[1])
    zr_gyro_list.append(gyro_means_init[2])

    # Append the rotated standard deviations of the gyroscope output to the list
    xr_gyro_stds_list.append(gyro_stds_init[0])
    yr_gyro_stds_list.append(gyro_stds_init[1])
    zr_gyro_stds_list.append(gyro_stds_init[2])

    # Create DataFrames from the list of gyroscope means and standard deviations
    gyro_df = pd.DataFrame({'xr_gyro': xr_gyro_list, 'yr_gyro': yr_gyro_list, 'zr_gyro': zr_gyro_list})
    gyro_stds_df = pd.DataFrame({'xr_gyro_std': xr_gyro_stds_list, 'yr_gyro_std': yr_gyro_stds_list, 'zr_gyro_std': zr_gyro_stds_list})
    
dob.reset()

Leg de dobbelsteen met nummer  1  boven
wacht 3 secondes
Logging data for 2s
Done! The data is located in self.datadf
Leg de dobbelsteen met nummer  2  boven
wacht 3 secondes
Logging data for 2s
Done! The data is located in self.datadf
Leg de dobbelsteen met nummer  3  boven
wacht 3 secondes
Logging data for 2s
Done! The data is located in self.datadf


In [6]:
# Calibration algorithm
# Remove the gravity for the bias of the accelerometer
xr_bias_acc = 1-np.abs(xr_acc_list[2])
yr_bias_acc = 1-np.abs(yr_acc_list[1])
zr_bias_acc = 1-np.abs(zr_acc_list[0])
bias = np.array([xr_bias_acc, yr_bias_acc, zr_bias_acc])
print("accelerometer bias = ", bias)

# Accelerometer standard deviations
xr_std = xr_acc_stds_list[2]
yr_std = yr_acc_stds_list[1]
zr_std = zr_acc_stds_list[0]
acc_std = np.array([xr_std, yr_std, zr_std])
print("accelerometer std = ", acc_stds)

# Biases for the gyroscope
xr_gyro_bias = np.mean(gyro_df['xr_gyro'])
yr_gyro_bias = np.mean(gyro_df['yr_gyro'])
zr_gyro_bias = np.mean(gyro_df['zr_gyro'])
gyro_bias = np.array([xr_gyro_bias,yr_gyro_bias,zr_gyro_bias])
print("gyroscope bias =", gyro_bias)

# Standard deviations for the gyroscope
xr_gyro_std = np.mean(gyro_stds_df['xr_gyro_std'])
yr_gyro_std = np.mean(gyro_stds_df['yr_gyro_std'])
zr_gyro_std = np.mean(gyro_stds_df['zr_gyro_std'])
gyro_std = np.array([xr_gyro_std, yr_gyro_std, zr_gyro_std])
print("gyroscope std =", gyro_std)

accelerometer bias =  [ 0.00783369  0.0221303  -0.05069003]
accelerometer std =  [0.00138887 0.00159395 0.00171181]
gyroscope bias = [-0.07755316  0.47813436 -0.05260948]
gyroscope std = [0.0226749  0.06621563 0.10385811]


In [14]:
print(data)

     timestamp     x_acc     y_acc     z_acc    x_gyro    y_gyro    z_gyro
0          0.0  0.750244 -0.026123 -0.656006 -0.076220  0.278201  0.064787
1         25.0  0.743896 -0.027344 -0.654297 -0.327744  0.342988  0.339177
2         30.0  0.745361 -0.024658 -0.655518 -0.175305  0.541159 -0.080030
3         35.0  0.750000 -0.025879 -0.659912  0.003811  0.506860  0.080030
4         40.0  0.749023 -0.024902 -0.662842 -0.083841  0.499238  0.060976
..         ...       ...       ...       ...       ...       ...       ...
383     1869.0  0.750000 -0.025635 -0.656250 -0.045732  0.575457 -0.022866
384     1875.0  0.748535 -0.023193 -0.653320  0.137195  0.453506 -0.038110
385     1880.0  0.750000 -0.023193 -0.654785 -0.141006  0.445884  0.049543
386     1886.0  0.747803 -0.024170 -0.655518 -0.209604  0.449695  0.072409
387     1890.0  0.747070 -0.022705 -0.656494 -0.232470  0.506860  0.022866

[388 rows x 7 columns]
